In [8]:
import os
import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup as bs
import json
import datetime as dt
import time

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
    

In [113]:
##################### DO NOT TOUCH THIS FUNCTION LIBERALLY ###########################

def pull_igtv(show_alias, use_case = 'show_title'):
    
    if use_case == 'selenium':
        soup = bs(show_alias, 'lxml')
    
    if use_case == 'specific_url':
        soup = bs(requests.get(show_alias).text, 'lxml')
        
    else:
        ## Site URL initialization
        site = 'https://www.instagram.com/{0}/channel/?hl=en'.format(show_alias)

        ## Requesting HTML content and storing in soup instance
        soup = bs(requests.get(site).text, 'lxml')

    ## Pulling overview of show, i.e., number of posts, followers, and following
    show_overview_info_metas = [str(meta) for meta in soup.find_all('meta')]
    filtered_metas = list(filter(lambda meta: 'Follow' in meta, show_overview_info_metas))[0].split('"')[1].split(' - ')[0]
    show_overview_summary = filtered_metas

    ## Extracting json element containing all post info
    raw_dict_string = str(soup.find_all('body')[0].find('script')).split('_sharedData = ')[1].replace(';</script>','')
    raw_json = json.loads(raw_dict_string)
    main_dict = raw_json['entry_data']['ProfilePage'][0]['graphql']['user']['edge_felix_video_timeline']
    
    ## Extracting number of posts from igtv show
    igtv_post_count = main_dict['count']
    
    ## Creating show data summary data frame by iterating
    ## over all posts and extracting necessary information
    igtv_dict = {'product_type':[],
                 'title':[],
                 'id':[],
                 'shortcode': [],
                 'owner_id': [],
                 'owner_username': [],
                 'snapshot_date_unix': [],
                 'video_duration_seconds': [],
                 'count_of_comments': [],
                 'number_of_likes': [],
                 'video_view_count': [],
                 'created_at': []}

    for n in main_dict['edges']:
        node = n['node']
        igtv_dict['product_type'].append(node['product_type'])
        igtv_dict['title'].append(node['title'])
        igtv_dict['id'].append(node['id'])
        igtv_dict['shortcode'].append(node['shortcode'])
        igtv_dict['owner_id'].append(node['owner']['id'])
        igtv_dict['owner_username'].append(node['owner']['username'])
        igtv_dict['snapshot_date_unix'].append(node['taken_at_timestamp'])
        igtv_dict['video_duration_seconds'].append(node['video_duration'])
        igtv_dict['count_of_comments'].append(node['edge_media_to_comment']['count'])
        igtv_dict['number_of_likes'].append(node['edge_liked_by']['count'])
        igtv_dict['video_view_count'].append(node['video_view_count'])
        igtv_dict['created_at'].append(node['created_at'])

    show_data_df = pd.DataFrame(igtv_dict)
    
    show_data_df['igtv_post_count'] = igtv_post_count
    show_data_df['show_overview_summary'] = show_overview_summary
    
    return show_data_df

In [4]:
##################### ADD SHOW NAMES TO LIST HERE #########################

show_aliases = ['fallontonight', 'agt', 'nbcsongland', 'ninjawarrior', 'nbcbluffcitylaw', 'nbcbringthefunny', 
                'nbccouncilofdads', 'nbcindebted', 'nbclincoln','nbcperfectharmony','nbcsongland','nbcsunnyside','nbctheinbetween',
               'nbckenanshow','zoeysplaylist','nbcsnl','nbcbrooklyn99','nbcworldofdance','fallontonight',
               'nbcthisisus','nbcthegoodplace','nbcsvu','latenightseth','nbcsuperstore','ninjawarrior']

#################### LOOP TO CREATE DATAFRAME OF ALL DATA ##############################

all_shows_df = []

for i, show in enumerate(show_aliases):
    curr_show_data_df = pull_igtv(show, use_selenium = False)
    all_shows_df.append(curr_show_data_df)
    print("Current Show: {0} --- Completed {1}%".format(show, round(i/len(show_aliases)*100,2)))
    time.sleep(30)

print("Completed 100%")

all_shows_df = pd.concat(all_shows_df, axis = 0).reset_index(drop = True)

currdate = dt.datetime.today().strftime('%Y-%m-%d')

################### SAVE TO CSV ############################
all_shows_df.to_csv('igtv_extract_{}.csv'.format(currdate))

Current Show: fallontonight --- Completed 0.0%
Current Show: agt --- Completed 20.0%
Current Show: nbcsongland --- Completed 40.0%
Current Show: ninjawarrior --- Completed 60.0%
Current Show: nbcbluffcitylaw --- Completed 80.0%
Completed 80.0%


## Dictionary codes for personal reference

product_type - product_type <br>
title - title <br>
id - id <br>
shortcode - shortcode <br>
owner - id <br>
owner - username <br>
taken_at_timestamp - unix time <br>
video_duration - in seconds <br>
edge_media_to_comment -- comments count <br>
edge_liked_by -- number of likes <br>
video_view_count - video_view_count <br>



In [102]:
def get_main_html_source(show_alias):

    site = 'https://www.instagram.com/{0}/channel/?hl=en'.format(show_alias)
    
    browser = webdriver.Chrome(executable_path=r"C:\Users\Fang\Desktop\Python Trading\Trading\chromedriver.exe")

    browser.get(site)

    source_list = []

    source_list.append(browser.page_source)

    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    while True:

        curr_page_source = browser.page_source
        source_list.append(curr_page_source)

        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    browser.quit()
    
    return source_list

In [98]:
def ig_pull_init_function(source):
    curr_post_divs = bs(source, 'lxml').find('main').find_all("a")

    post_init_info = {'post_title': [],
                      'post_url': []}

    for a in curr_post_divs:
        href = a['href'] 
        text = a.text 
        if '/tv/' in href:
            post_init_info['post_url'].append('https://www.instagram.com{}'.format(href))
            post_init_info['post_title'].append(text)

    return pd.DataFrame(post_init_info)
    
    

In [168]:
def get_post_info(curr_url):

    curr_post = bs(requests.get(curr_url).text, 'lxml')

    curr_post_json = json.loads(curr_post.find('script', type = 'application/ld+json').text.strip())

    alternate_name = curr_post_json['author']['alternateName']
    commentCount = curr_post_json['commentCount']
    
    try:
        caption = curr_post_json['caption']
    except:
        caption = ''
        
    likes = curr_post_json['description'].split(' Likes, ')[0].replace(',', '')
    views = curr_post_json['interactionStatistic']['userInteractionCount']
    uploadDate = curr_post_json['uploadDate']

    curr_post_df = pd.DataFrame({'alternate_name': alternate_name,
                                 'commentCount': commentCount,
                                 'caption': caption,
                                 'likes': likes,
                                 'views': views,
                                 'uploadDate': uploadDate}, index = [0])
    return curr_post_df

In [172]:
def get_show_info(show_alias):

    source_list = get_main_html_source(show_alias)

    source_post_links_df = pd.concat([ig_pull_init_function(source) for source in source_list], axis = 0).drop_duplicates().reset_index(drop = True)

    show_df = []

    for idx, row in source_post_links_df.iterrows():
        post_title = row['post_title'].split(':')
        duration = '{0}:{1}'.format(post_title[0], post_title[1][:2])
        title = post_title[1][2:]

        curr_url = row['post_url']

        curr_post_df = get_post_info(curr_url)

        curr_post_df['duration'] = duration
        curr_post_df['title'] = title

        show_df.append(curr_post_df)

        print("Current Show: {0} --- Completed {1}%".format(title, round(i/len(show_aliases)*100,2)))
        time.sleep(5)

    show_df = pd.concat(show_df, axis = 0).reset_index(drop = True)
    
    return show_df

author[alternateName] - alternate name 
commentCount - number of comments
caption - caption
description - description
interactionStatistic[userInteractionCount] - views
uploadDate

